# Combine ESPN and Pro Football Reference Dataframes
## By: Nick Bruno

### Import libraries

In [1]:
import pandas as pd
import os
import numpy as np

### Import datasets

##### Import QBR dataset

In [2]:
qbr_data = pd.read_csv('regular_season_qbr_2006_to_2018.csv', index_col=None)

In [3]:
# Drop index column
qbr_data = qbr_data.drop(columns='Unnamed: 0')

In [4]:
# List columns
qbr_data.columns

Index(['PASS', 'PENALTY', 'PLAYER', 'PTS ADDED', 'QB PLAYS', 'RAW QBR',
       'RESULT', 'RK', 'RUN', 'TOTAL EPA', 'TOTAL QBR', 'WEEK', 'YEAR'],
      dtype='object')

In [5]:
# Checkout first 5 rows
qbr_data.head()

,PASS,PENALTY,PLAYER,PTS ADDED,QB PLAYS,RAW QBR,RESULT,RK,RUN,TOTAL EPA,TOTAL QBR,WEEK,YEAR
0,4.3,0.1,"Rex Grossman, CHI",5.5,31,92.4,W 26-0 @ GB in Wk 1,1.0,0.6,5.4,96.4,1.0,2006.0
1,8.6,-0.1,"Donovan McNabb, PHI",8.1,38,95.4,W 24-10 @ HOU in Wk 1,2.0,0.1,8.9,92.5,1.0,2006.0
2,6.5,0.2,"Chad Pennington, NYJ",6.1,43,88.1,W 23-16 @ TEN in Wk 1,3.0,-0.3,7.5,90.1,1.0,2006.0
3,8.4,-0.3,"Peyton Manning, IND",5.8,45,86.0,W 26-21 @ NYG in Wk 1,4.0,0.0,8.3,86.0,1.0,2006.0
4,1.3,0.4,"Mike Vick, ATL",1.8,30,70.5,W 20-6 @ CAR in Wk 1,5.0,1.1,3.4,79.4,1.0,2006.0


In [6]:
# Shape
qbr_data.shape

(6537, 13)

##### Import pro-football-reference quarterback dataset

In [7]:
pfr_data = pd.read_csv('qb_data.csv')

In [8]:
# List columns
pfr_data.columns

Index(['name', 'id', 'Year', 'Date', 'G.', 'Age', 'Tm', 'X', 'Opp', 'Result',
       'GS', 'Cmp', 'Att', 'Cmp.', 'Yds', 'TD', 'Int', 'Rate', 'Sk', 'Yds.1',
       'Y.A', 'AY.A', 'Att.1', 'Yds.2', 'Y.A.1', 'TD.1', 'TD.2', 'Pts', 'Fmb',
       'FF', 'FR', 'Yds.3', 'TD.3', 'team_win', 'start_win', 'game_count',
       'start_count'],
      dtype='object')

In [9]:
# Checkout first 5 rows
pfr_data.head()

,name,id,Year,Date,G.,Age,Tm,X,Opp,Result,...,Pts,Fmb,FF,FR,Yds.3,TD.3,team_win,start_win,game_count,start_count
0,A.J. Feeley,FeelA.00,2001,1/6/02,16,24.235,PHI,@,TAM,W 17-13,...,0,0,0,0,0,0,1,0,1,0
1,A.J. Feeley,FeelA.00,2002,12/1/02,12,25.199,PHI,NaN,STL,W 10-3,...,0,0,0,0,0,0,1,1,2,1
2,A.J. Feeley,FeelA.00,2002,12/8/02,13,25.206,PHI,@,SEA,W 27-20,...,0,0,0,0,0,0,1,1,3,2
3,A.J. Feeley,FeelA.00,2002,12/15/02,14,25.213,PHI,NaN,WAS,W 34-21,...,0,0,0,0,0,0,1,1,4,3
4,A.J. Feeley,FeelA.00,2002,12/21/02,15,25.219,PHI,@,DAL,W 27-3,...,0,0,0,0,0,0,1,1,5,4


We can tell that the pro-football-reference.com data contains more data compared to the ESPN data. This will be helpful when conducting analysis. For now, we are trying to merge the espn.com and pro-football-reference.com, so it is important to find which data is included in both dataframes and how this data can be used to correctly identify individual games, leading to an eventual natural join between the dataframes.

### Subset 'pfr_data' to only include statistics from years 2006 to 2018

The data gathered from espn.com only includes data starting from the 2006 NFL season, so we should subset the pro-football-reference.com data to only include data starting in 2006.

In [14]:
pfr_data = pfr_data[pfr_data.Year > 2005]

In [15]:
pfr_data.Year.unique() # dataframe contains data from years 2006 to 2018

array([2006, 2007, 2011, 2015, 2008, 2009, 2010, 2012, 2013, 2014, 2016,
       2017, 2018])

In [16]:
pfr_data.shape

(6453, 37)

## Feature Generation

Because there is no common column between the two dataframes, we will have to get creative in combining them. My initial thought is to merge them by name, year, and result. This will require some column manipulation, which I will demonstrate in the following code.

### Name column

In [17]:
# First, edit the qbr_data 'PLAYER' column to a 'name' column and a 'team' column
qbr_data['name'] = qbr_data.PLAYER.str.split(',').str[0]
qbr_data['Tm'] = qbr_data.PLAYER.str.split(',').str[1].str.lstrip() # strip initial whitespace

In [18]:
qbr_data[['name','Tm']].head()

,name,Tm
0,Rex Grossman,CHI
1,Donovan McNabb,PHI
2,Chad Pennington,NYJ
3,Peyton Manning,IND
4,Mike Vick,ATL


##### Compare QB names in both dataframes

In [19]:
# Get a list of each QB name from both dataframes
qbr_names = qbr_data.name.unique()
pfr_names = pfr_data.name.unique()

In [20]:
len(qbr_names)

191

In [21]:
len(pfr_names)

171

Here, we can see that espn.com dataframe contains 20 more quarterbacks compared to pro-football-reference.com. Now, its important to find a list of common quarterbacks between the dataframes, identify potential name differences in the dataframes, and understand why a quarterback may be excluded from one dataframe and not the other.

##### Identify quarterback names that occur in both dataframes

In [22]:
common_names = list(set(qbr_names).intersection(pfr_names))

In [23]:
len(common_names) # 160, 11 less than the number of QB names in the pfr dataframe

160

##### Identify QBs in pfd dataframe not in espn dataframe

In [24]:
pfr_diff = np.setdiff1d(pfr_names, common_names)
pfr_diff # most of these names are slightly different in the qbr dataset

array(['A.J. McCarron', 'Alex Tanney', 'Brett Basanez', 'Brett Favre+',
       'Craig Nall', 'Kurt Warner+', 'Michael Vick', 'Mitch Trubisky',
       'Robert Griffin', 'Terrelle Pryor', 'Tony Pike'], dtype=object)

Some of the names are slightly different in the datasets. For example, the qbr_data uses "Mike Vick" rather than "Michael Vick". I will fix these issues. Also, pfr_data adds a '+' symbol to the names of hall of fame quarterbacks. That will also be adjusted.

In [25]:
# Change hall of fame quarterbacks to exclude the '+' symbol
pfr_data['name'] = pfr_data['name'].replace('Brett Favre+', 'Brett Favre')
pfr_data['name'] = pfr_data['name'].replace('Kurt Warner+', 'Kurt Warner')

In [26]:
# Replace Names in pfr dataframe to match espn dataframe
pfr_data['name'] = pfr_data['name'].replace('Michael Vick', 'Mike Vick')
pfr_data['name'] = pfr_data['name'].replace('Robert Griffin', 'Robert Griffin III')
pfr_data['name'] = pfr_data['name'].replace('Mitch Trubisky', 'Mitchell Trubisky')
pfr_data['name'] = pfr_data['name'].replace('A.J. McCarron', 'AJ McCarron')
pfr_data['name'] = pfr_data['name'].replace('Terrelle Pryor', 'Terrelle Pryor Sr.')

Now that these changes have been made, the list of 'common names' should increase.

In [27]:
# Edit lists to account for adjustments in pfr_data
qbr_names = qbr_data.name.unique()
pfr_names = pfr_data.name.unique()
common_names_mod = list(set(qbr_names).intersection(pfr_names))
len(common_names_mod) # Now 167 common QBs

167

In [28]:
# Find the quarterbacks in the ESPN data that are not in the pro-football-reference data
np.setdiff1d(qbr_names, common_names_mod)

array(['Brian St. Pierre', 'Charlie Whitehurst', "J.T. O'Sullivan",
       'Jameis Winston', 'Jeff Driskel', 'Joe Webb III', 'Josh Johnson',
       'Josh McCown', 'Josh Rosen', 'Kellen Moore', 'Kyle Allen',
       'Lamar Jackson', 'Landry Jones', 'Matt McGloin', 'Matt Moore',
       'Nick Mullens', 'Russell Wilson', 'Rusty Smith', 'T.J. Yates',
       'Taylor Heinicke', 'Thaddeus Lewis', 'Trent Edwards', 'Troy Smith',
       'Vince Young'], dtype=object)

There are some quarterbacks that are in the ESPN QBR dataset that are not in the pro-football-reference dataset. That is likely a result in the collection of the pro-football-reference Quarterback data. Although there are some big name quarterbacks excluded from the data (Russell Wilson, Jameis Winston, Vince Young), we will continue with the analysis with the 167 common quarterbacks in both datasets.

In [29]:
# Edit dataframes to only include data on these quarterbacks
pfr_data_mod1 = pfr_data[pfr_data.name.isin(common_names_mod)]

In [30]:
qbr_data_mod1 = qbr_data[qbr_data.name.isin(common_names_mod)]

In [31]:
pfr_data_mod1.shape[0]

6449

In [32]:
qbr_data_mod1.shape[0]

6094

pro-football-reference subset contains 400 more games compared to the ESPN data. This is likely because QBR is not calculated for each quarterback every week on the ESPN website. Hopefully, however, the merged dataframe will contain close to the 6,094 rows in the qbr_data_mod1 dataframe.

### Mark home (vs) and away (@) games in both dataframes

##### Check both dataframes

In [33]:
# Check pfr dataframe
pfr_data_mod1.head(1)

,name,id,Year,Date,G.,Age,Tm,X,Opp,Result,...,Pts,Fmb,FF,FR,Yds.3,TD.3,team_win,start_win,game_count,start_count
16,A.J. Feeley,FeelA.00,2006,12/31/06,16,29.229,PHI,NaN,ATL,W 24-17,...,0,0,0,0,0,0,1,0,17,13


In [34]:
# Check espn dataframe
qbr_data_mod1.head(1)

,PASS,PENALTY,PLAYER,PTS ADDED,QB PLAYS,RAW QBR,RESULT,RK,RUN,TOTAL EPA,TOTAL QBR,WEEK,YEAR,name,Tm
0,4.3,0.1,"Rex Grossman, CHI",5.5,31,92.4,W 26-0 @ GB in Wk 1,1.0,0.6,5.4,96.4,1.0,2006.0,Rex Grossman,CHI


pfr_data_mod1 marks home or away in the 'X' column, while qbr_data_mod1 marks this in 'RESULT' column

In [35]:
# pfr_dat_mod1 
pfr_data_mod1.X.unique()

array([nan, '@', 'N'], dtype=object)

In [36]:
# Investigate what 'N' means
pfr_data_mod1[pfr_data_mod1.X == 'N'].iloc[:2] # these represent superbowl games ('N' means neutral)

,name,id,Year,Date,G.,Age,Tm,X,Opp,Result,...,Pts,Fmb,FF,FR,Yds.3,TD.3,team_win,start_win,game_count,start_count
275,Aaron Rodgers,RodgAa00,2010,2/6/11,20,27.066,GNB,N,PIT,W 31-25,...,0,0,0,0,0,0,1,1,156,153
1457,Ben Roethlisberger,RoetBe00,2008,2/1/09,19,26.336,PIT,N,ARI,W 27-23,...,0,0,0,0,0,0,1,1,218,216


In [37]:
pfr_data_mod1[pfr_data_mod1.X.isnull()].iloc[:2] # represents home games

,name,id,Year,Date,G.,Age,Tm,X,Opp,Result,...,Pts,Fmb,FF,FR,Yds.3,TD.3,team_win,start_win,game_count,start_count
16,A.J. Feeley,FeelA.00,2006,12/31/06,16,29.229,PHI,NaN,ATL,W 24-17,...,0,0,0,0,0,0,1,0,17,13
17,A.J. Feeley,FeelA.00,2007,11/18/07,10,30.186,PHI,NaN,MIA,W 17-7,...,0,0,0,0,0,0,1,0,18,13


In [38]:
# Replace null values with vs
pfr_data_mod1['X'] = pfr_data_mod1['X'].fillna('vs')

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [39]:
# qbr_data_mod1
# Create an 'X' column like in pfr_data_mod1
qbr_data_mod1['X'] = "" # initiate new column
qbr_data_mod2 = qbr_data_mod1.assign(X=qbr_data_mod1.RESULT.str.contains('@').map({True: '@', False: 'vs'}))

# https://stackoverflow.com/questions/43290762/how-to-check-pandas-dataframe-for-true-or-false-python

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


### RESULT column split to show scores and opponent

Now, we want to split up the RESULT column in qbr_data_mod2 to separately display the score and opponent in the same manner for both dataframes.

In [40]:
# First, to make it easier to follow, I will set a new dataframe, pfr_data_mod2 equal to pfr_data_mod1.
pfr_data_mod2 = pfr_data_mod1

Because we already have the 'X' column, I will go ahead and replace the 'vs' in RESULTS with '@' so that there is a consistent delimiter, making it easier to split the RESULT column

In [41]:
# Replace 'vs' with '@' in qbr_data_mod2 to make it easier to split the score and game result
qbr_data_mod2.RESULT = qbr_data_mod2.RESULT.str.replace('vs','@')

### Expand RESULT and Score columns to [Result, Hm_score, Away_score]

Important note: espn.com presents the score with the highest score (winning team) presented first while pfr.com presents the score with the winning quarterback's team first.

##### qbr

In [42]:
qbr_data_mod2.RESULT.iloc[9]

'L 19-17  @ NE in Wk 1'

In [43]:
qbr_data_mod2['Game_result'] = qbr_data_mod2.RESULT.str.split('@').str[0].str.split(' ').str[0]

In [44]:
qbr_data_mod2.Game_result.unique() # win, lose, or draw

array(['W', 'L', 'T'], dtype=object)

In [45]:
def home_score(string):
    if string[0] == 'W':
        home_score = int(string.strip().split('-')[0].split(' ')[1])
    elif string[0] == 'L':
        home_score = int(string.split('-')[1].split('@')[0].strip())
    elif string[0] == 'T':
        home_score = int(string.strip().split('-')[0].split(' ')[1]) # doesn't matter, both scores are same
    else:
        home_score = 'NA'
    return home_score

In [46]:
def away_score(string):
    if string[0] == 'W':
        away_score = int(string.split('-')[1].split('@')[0].strip())
    elif string[0] == 'L':
        away_score = int(string.strip().split('-')[0].split(' ')[1])
    elif string[0] == 'T':
        away_score = int(string.strip().split('-')[0].split(' ')[1])
    else:
        away_score = 'NA'
    return away_score

In [47]:
qbr_data_mod2['Home_score'] = qbr_data_mod2.RESULT.apply(home_score)

In [48]:
qbr_data_mod2['Away_score'] = qbr_data_mod2.RESULT.apply(away_score)

In [49]:
# Split 'RESULT' to only show the Opponent
qbr_data_mod2['Opp'] = qbr_data_mod2.RESULT.str.split('@').str[1].str.split('in').str[0].str.strip()

##### pfr

In [50]:
pfr_data_mod2.Result.iloc[:2]

16    W 24-17
17     W 17-7
Name: Result, dtype: object

In [51]:
pfr_data_mod2['Game_result'] = pfr_data_mod2.Result.str.split('-').str[0].str.strip().str.split(' ').str[0]

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [52]:
pfr_data_mod2['Home_score'] = pfr_data_mod2.Result.str.split('-').str[0].str.strip().str.split(' ').str[1].astype(int)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [53]:
pfr_data_mod2['Away_score'] = pfr_data_mod2.Result.str.split('-').str[1].str.strip().astype(int)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [54]:
qbr_data_mod2[['Game_result','Home_score','Away_score','Opp']].head(2)

,Game_result,Home_score,Away_score,Opp
0,W,26,0,GB
1,W,24,10,HOU


We will use these four columns to help merge.

### Investigate Team name abbreviations

##### Change 'Opp' columns to share the same abbreviations

In [55]:
# Get unique list of team abbreviations
qbr_opp = qbr_data_mod2.Opp.unique()
pfr_opp = pfr_data_mod2.Opp.unique()

In [56]:
qbr_opp

array(['GB', 'HOU', 'TEN', 'NYG', 'CAR', 'MIA', 'TB', 'DAL', 'WSH', 'NE',
       'MIN', 'IND', 'JAX', 'BUF', 'CLE', 'PHI', 'ARI', 'SF', 'SEA', 'NO',
       'KC', 'DET', 'CHI', 'PIT', 'CIN', 'LAC', 'NYJ', 'BAL', 'ATL',
       'LAR', 'OAK', 'DEN'], dtype=object)

In [57]:
pfr_opp

array(['ATL', 'MIA', 'NWE', 'SEA', 'DAL', 'NOR', 'SFO', 'PIT', 'DEN',
       'BAL', 'SDG', 'KAN', 'HOU', 'CIN', 'STL', 'NYJ', 'MIN', 'DET',
       'TAM', 'IND', 'TEN', 'CHI', 'CAR', 'JAX', 'CLE', 'ARI', 'PHI',
       'BUF', 'WAS', 'NYG', 'OAK', 'LAR', 'GNB', 'LAC'], dtype=object)

pro-football-reference uses 3 letter abbreviations for each team. ESPN, however, uses the regular abbreviations. I will change the pro-football-reference team abbreviations to match ESPN's.

In [58]:
len(qbr_opp)

32

In [59]:
len(pfr_opp)

34

In [60]:
np.setdiff1d(qbr_opp, pfr_opp)

array(['GB', 'KC', 'NE', 'NO', 'SF', 'TB', 'WSH'], dtype=object)

In [61]:
# Replace the incorrect abbreviations in pro-football-reference data for the 'Opp' column
pfr_data_mod2['Opp'] = pfr_data_mod2['Opp'].replace('GNB', 'GB')
pfr_data_mod2['Opp'] = pfr_data_mod2['Opp'].replace('KAN', 'KC')
pfr_data_mod2['Opp'] = pfr_data_mod2['Opp'].replace('NWE', 'NE')
pfr_data_mod2['Opp'] = pfr_data_mod2['Opp'].replace('NOR', 'NO')
pfr_data_mod2['Opp'] = pfr_data_mod2['Opp'].replace('SFO', 'SF')
pfr_data_mod2['Opp'] = pfr_data_mod2['Opp'].replace('TAM', 'TB')
pfr_data_mod2['Opp'] = pfr_data_mod2['Opp'].replace('WAS', 'WSH')

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy o

In [62]:
np.setdiff1d(pfr_opp, qbr_opp)

array(['GNB', 'KAN', 'NOR', 'NWE', 'SDG', 'SFO', 'STL', 'TAM', 'WAS'],
      dtype=object)

There are two extra abbreviations in the pro-football-reference data. This is because the San Diego Chargers and St. Louis Rams both moved to Los Angeles. This is reflected in the ESPN data, as all of the old teams were replaced with the new abbreviations. I will make the changes to the pro-football-reference data.

In [63]:
pfr_data_mod2['Opp'] = pfr_data_mod2['Opp'].replace('STL', 'LAR')
pfr_data_mod2['Opp'] = pfr_data_mod2['Opp'].replace('SDG', 'LAC')

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [64]:
# Make the same changes for the 'Tm' Column of pfr_data_mod2
pfr_data_mod2['Tm'] = pfr_data_mod2['Tm'].replace('GNB', 'GB')
pfr_data_mod2['Tm'] = pfr_data_mod2['Tm'].replace('KAN', 'KC')
pfr_data_mod2['Tm'] = pfr_data_mod2['Tm'].replace('NWE', 'NE')
pfr_data_mod2['Tm'] = pfr_data_mod2['Tm'].replace('NOR', 'NO')
pfr_data_mod2['Tm'] = pfr_data_mod2['Tm'].replace('SFO', 'SF')
pfr_data_mod2['Tm'] = pfr_data_mod2['Tm'].replace('TAM', 'TB')
pfr_data_mod2['Tm'] = pfr_data_mod2['Tm'].replace('WAS', 'WSH')
pfr_data_mod2['Tm'] = pfr_data_mod2['Tm'].replace('STL', 'LAR')
pfr_data_mod2['Tm'] = pfr_data_mod2['Tm'].replace('SDG', 'LAC')

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy o

We should now have enough matching columns that we can merge the dataframes.

### Clean columns for dataframe merging

In [65]:
# List qbr columns
qbr_columns = qbr_data_mod2.columns
qbr_columns

Index(['PASS', 'PENALTY', 'PLAYER', 'PTS ADDED', 'QB PLAYS', 'RAW QBR',
       'RESULT', 'RK', 'RUN', 'TOTAL EPA', 'TOTAL QBR', 'WEEK', 'YEAR', 'name',
       'Tm', 'X', 'Game_result', 'Home_score', 'Away_score', 'Opp'],
      dtype='object')

In [66]:
# List pfr columns
pfr_columns = pfr_data_mod2.columns
pfr_columns

Index(['name', 'id', 'Year', 'Date', 'G.', 'Age', 'Tm', 'X', 'Opp', 'Result',
       'GS', 'Cmp', 'Att', 'Cmp.', 'Yds', 'TD', 'Int', 'Rate', 'Sk', 'Yds.1',
       'Y.A', 'AY.A', 'Att.1', 'Yds.2', 'Y.A.1', 'TD.1', 'TD.2', 'Pts', 'Fmb',
       'FF', 'FR', 'Yds.3', 'TD.3', 'team_win', 'start_win', 'game_count',
       'start_count', 'Game_result', 'Home_score', 'Away_score'],
      dtype='object')

In [67]:
# Change 'YEAR' to 'Year' in qbr_data_mod2
qbr_data_mod2 = qbr_data_mod2.rename(columns={'YEAR':'Year'})

In [68]:
# Common columns
common_cols = list(set(pfr_data_mod2.columns).intersection(qbr_data_mod2.columns))
common_cols

['name', 'Game_result', 'Year', 'Away_score', 'Opp', 'X', 'Tm', 'Home_score']

In [69]:
# Change Year float to int in qbr
qbr_data_mod2['Year'] = qbr_data_mod2['Year'].astype(int)

Now we have enough similar columns to merge the dataframes.

### Merge without scores

In [70]:
old_df = pd.merge(left=qbr_data_mod2, right=pfr_data_mod2, on=['name','Year','Opp','X'], how='inner')

In [71]:
old_df.shape

(5979, 56)

In [72]:
old_df['Home_score_diff'] = old_df.Home_score_x - old_df.Home_score_y

In [73]:
old_df[['name','Year','Tm_x','Tm_y','Opp','X','Home_score_diff','Home_score_x','Home_score_y','Away_score_y',
        'Away_score_x','Game_result_x','Game_result_y']] \
[(old_df.Home_score_diff != 0)]

,name,Year,Tm_x,Tm_y,Opp,X,Home_score_diff,Home_score_x,Home_score_y,Away_score_y,Away_score_x,Game_result_x,Game_result_y
40,Eli Manning,2006,NYG,NYG,PHI,@,10,30,20,23,24,W,L
96,Rex Grossman,2006,CHI,CHI,SEA,vs,10,37,27,24,6,W,W
112,Matt Hasselbeck,2006,SEA,SEA,CHI,@,-18,6,24,27,37,L,L
261,Tom Brady,2006,NE,NE,NYJ,vs,-23,14,37,16,17,L,W
263,Chad Pennington,2006,NYJ,NYJ,NE,@,1,17,16,37,14,W,L
489,Tony Romo,2007,DAL,DAL,NYG,vs,28,45,17,21,35,W,L
491,Eli Manning,2007,NYG,NYG,DAL,@,14,35,21,17,45,L,W
520,Tom Brady,2007,NE,NE,LAC,vs,17,38,21,12,14,W,W
545,Philip Rivers,2007,LAC,LAC,NE,@,2,14,12,21,38,L,L
873,David Garrard,2007,JAX,JAX,PIT,@,-2,29,31,29,22,W,W


In [74]:
old_df[['name','Year','Tm_x','Tm_y','Opp','X','Home_score_diff','Home_score_x','Home_score_y','Away_score_y',
        'Away_score_x','Game_result_x','Game_result_y']] \
[(old_df.name == 'Eli Manning')].head()

,name,Year,Tm_x,Tm_y,Opp,X,Home_score_diff,Home_score_x,Home_score_y,Away_score_y,Away_score_x,Game_result_x,Game_result_y
11,Eli Manning,2006,NYG,NYG,IND,vs,0,21,21,26,26,L,L
39,Eli Manning,2006,NYG,NYG,PHI,@,0,30,30,24,24,W,W
40,Eli Manning,2006,NYG,NYG,PHI,@,10,30,20,23,24,W,L
88,Eli Manning,2006,NYG,NYG,SEA,@,0,30,30,42,42,L,L
126,Eli Manning,2006,NYG,NYG,WSH,vs,0,19,19,3,3,W,W


Because the pfr.com dataframe includes games from the playoffs, there are some inconsistencies when the dataframes are merged without the Home and away scores. So, when we merge, we expect 5,979 rows.

### Correct Merge

In [75]:
common_cols

['name', 'Game_result', 'Year', 'Away_score', 'Opp', 'X', 'Tm', 'Home_score']

In [76]:
qbr_sub = qbr_data_mod2[common_cols]

In [77]:
pfr_sub = pfr_data_mod2[common_cols]

In [78]:
pfr_sub.Tm = pfr_sub.Tm.str.lstrip()

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/generic.py:4401: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [85]:
merge_df = pd.merge(left=qbr_data_mod2, right=pfr_data_mod2, on=common_cols, how='inner')

In [86]:
merge_df.shape

(5899, 52)

##### List all columns available in the merged dataframe

In [88]:
merge_df.columns

Index(['PASS', 'PENALTY', 'PLAYER', 'PTS ADDED', 'QB PLAYS', 'RAW QBR',
       'RESULT', 'RK', 'RUN', 'TOTAL EPA', 'TOTAL QBR', 'WEEK', 'Year', 'name',
       'Tm', 'X', 'Game_result', 'Home_score', 'Away_score', 'Opp', 'id',
       'Date', 'G.', 'Age', 'Result', 'GS', 'Cmp', 'Att', 'Cmp.', 'Yds', 'TD',
       'Int', 'Rate', 'Sk', 'Yds.1', 'Y.A', 'AY.A', 'Att.1', 'Yds.2', 'Y.A.1',
       'TD.1', 'TD.2', 'Pts', 'Fmb', 'FF', 'FR', 'Yds.3', 'TD.3', 'team_win',
       'start_win', 'game_count', 'start_count'],
      dtype='object')

In [90]:
# Take a peak at the columns we are most interested in
merge_df[['name', 'TOTAL QBR','Rate','WEEK','Year','Tm']].head(2)

,name,TOTAL QBR,Rate,WEEK,Year,Tm
0,Rex Grossman,96.4,98.6,1.0,2006,CHI
1,Donovan McNabb,92.5,113.3,1.0,2006,PHI


##### Write merged dataframe to a .csv

In [91]:
merge_df.to_csv('correct_merge_qb_ratings_df.csv', index=False, header=True)

We now have a merged dataframe that now adds the QBR statistic from the espn.com dataset to the extensive statistics avaiable in the pro-football-reference.com dataset. This dataset contains 5,899 rows and 52 columns of data which represent an NFL quarterback's statistics for one game from the 2006 to 2018 NFL seasons. This merged dataframe will be used for future blog posts with the goal of "reverse engineering" the algorithm that ESPN uses to calculate quarterback rating.